1. Aritra Banerjee
2. Aishwarya R

# Importing Packages

In [24]:
from sklearn.metrics import f1_score, precision_score, recall_score, accuracy_score
from sklearn.feature_extraction.text import TfidfVectorizer
from mlxtend.preprocessing import TransactionEncoder
from sklearn.model_selection import cross_val_score
from sklearn.multiclass import OneVsRestClassifier
from sklearn.naive_bayes import MultinomialNB
from bs4 import BeautifulSoup,SoupStrainer
from sklearn.pipeline import Pipeline
import pandas as pd
import numpy as np
import warnings
import time
import re
warnings.filterwarnings("ignore")

f = open("output_of_naive_bayes.txt",'w')

# Parsing Data (Creating Test Train) 

In [2]:
start_time_parsing = time.clock()

In [3]:
train = []
test = []
train_category = []
test_category = []
lis = ["0" + str(i) for i in range(10)] + list(range(10,22))
for file_name in lis:
    f = open('data/reut2-0'+str(file_name)+'.sgm', 'r')
    data= f.read()
    soup = BeautifulSoup(data)
    contents_train = soup.find_all('reuters', lewissplit="TRAIN") or soup.find_all('reuters', lewissplit="NOT-USED") 
    #contents_train = soup.find_all('reuters', cgisplit="TRAINING-SET")
    
    for doc_train in range(0,len(contents_train)):
        try:
            temp_train = []
            for node in contents_train[doc_train].topics.find_all('d'):
                temp_train.append(''.join(node.findAll(text=True)))
            if temp_train[0] != '':
                if contents_train[doc_train].find('body') != None:
                    train.append(contents_train[doc_train].title.text + contents_train[doc_train].body.string)
                    train_category.append(temp_train)
                elif contents_train[doc_train].find('text', type = 'UNPROC') != None:
                    train.append(contents_train[doc_train].find('text', type = 'UNPROC').text)
                    train_category.append(temp_train)
                elif contents_train[doc_train].find('text', type = 'BRIEF') != None:
                    train.append(contents_train[doc_train].find('text', type = 'BRIEF').title.text)
                    train_category.append(temp_train)
        except:
            continue
         
         
    contents_test = soup.find_all('reuters', lewissplit="TEST")  
    #contents_test = soup.find_all('reuters', cgisplit="PUBLISHED-TESTSET")
    for doc_test in range(0,len(contents_test)):
        try:
            temp_test = []
            for node in contents_test[doc_test].topics.find_all('d'):
                temp_test.append(''.join(node.findAll(text=True)))
            if temp_test[0] != '':
                if contents_test[doc_test].find('body') != None:
                    test.append(contents_test[doc_test].title.text + contents_test[doc_test].body.string)
                    test_category.append(temp_test)
                elif contents_test[doc_test].find('text', type = 'UNPROC') != None:
                    test.append(contents_test[doc_test].find('text', type = 'UNPROC').text)
                    test_category.append(temp_test)
                elif contents_test[doc_test].find('text', type = 'BRIEF') != None:
                    test.append(contents_test[doc_test].find('text', type = 'BRIEF').title.text)
                    test_category.append(temp_test)
        except:
            continue

In [4]:
end_time_parsing = time.clock()
time_parsing = (end_time_parsing-start_time_parsing)

# Modifying Datastructure

In [5]:
# train_data = pd.DataFrame(train)
# train_category_data = pd.DataFrame(train_category)
# test_data = pd.DataFrame(test)
# test_category_data = pd.DataFrame(test_category)

In [6]:
all_train_cat = []
for i in train_category:
    all_train_cat = all_train_cat + i
    
all_test_cat = []
for i in test_category:
    all_test_cat = all_test_cat + i

not_in_train = set(all_test_cat).difference(set(all_train_cat))

In [7]:
idx = []
for i in range(len(test_category)):
    if not_in_train.intersection(set(test_category[i])) != set():
        idx.append(i) 

In [8]:
count = 0
for i in idx:
    train.append(test[i-count])
    train_category.append(test_category[i-count])
    test.pop(i-count)
    test_category.pop(i-count)
    count += 1

In [9]:
data = zip(train,train_category)
data = pd.DataFrame(data,columns=("body","label"))

data.drop_duplicates(subset="body",inplace=True)

data.describe()

,body,label
count,7827,7827
unique,7827,492
top,PHILIPPINE COCONUT INDUSTRY WORRIED BY EC TAXP...,[earn]
freq,1,2837


In [10]:
train = list(data.body)
train_category = list(data.label)

In [11]:
invalid = re.compile(r"[\t\n\/\x03\,\.\"\']+") 
def map_all(doc):
    return(invalid.sub(' ', doc))

invalid_1 = re.compile(r'\b[\d.]*[\d]+\b')
def map_all_2(doc):
    return(invalid_1.sub(' ', doc))

train = map(map_all,train)
train = map(map_all_2,train)
test = map(map_all,test)
test = map(map_all_2,test)

# Processing the data for fitting

In [12]:
 start_time_model = time.clock()

In [13]:
te = TransactionEncoder()
train_array = te.fit(train_category).transform(train_category)
train_matrix = pd.SparseDataFrame(train_array, columns=te.columns_, default_fill_value=0)

In [14]:
test_array = te.fit(train_category).transform(test_category)
test_matrix = pd.SparseDataFrame(test_array, columns=te.columns_, default_fill_value=0)

In [15]:
vectorizer = TfidfVectorizer(max_features = 5000)

# Training & Predicting

In [16]:
vectorised_train_documents = vectorizer.fit_transform(train)
vectorised_test_documents = vectorizer.transform(test)

In [17]:
classifier = OneVsRestClassifier(MultinomialNB(fit_prior=True, class_prior=None, alpha=0.01))
classifier.fit(vectorised_train_documents, train_array)

OneVsRestClassifier(estimator=MultinomialNB(alpha=0.01, class_prior=None, fit_prior=True),
          n_jobs=None)

In [18]:
predictions = classifier.predict(vectorised_test_documents)

In [19]:
end_time_model = time.clock()
time_model = (end_time_model - start_time_model) 

# Evaluating Model

In [20]:
precision = precision_score(test_array, predictions,average='micro')
#recall = recall_score(test_array, predictions,average='micro')
f1 = f1_score(test_array, predictions, average='micro')

accuracy = accuracy_score(predictions, test_array)

# Mapping prediction to the category

In [21]:
predicted_category = []
for j in predictions:
    res = [i for i, val in enumerate(j) if val]
    temp = []
    for k in res:
        temp.append(test_matrix.columns[k])
    predicted_category.append(temp)

In [22]:
output = zip(test_category, predicted_category)
output = pd.DataFrame(output)

# Printing

In [25]:
f.write("Output of Naive Bayes::::  ")
f.write("\n")
f.write("\n")

len_train = str(len(train))
len_test = str(len(test))
f.write("Length of the training data is::::  ")
f.write(len_train)
f.write("\n")
f.write("Length of the test data is::::  ")
f.write(len_test)
f.write("\n")
f.write("\n")


f.write("Micro-average quality numbers::::  ")
f.write("Precision::::  ")
f.write(str(precision))
f.write("\n")
f.write("F1 score::::  ")
f.write(str(f1))
f.write("\n")        
f.write("Accuracy-score::::  ")
f.write(str(accuracy))
f.write("\n")
f.write("\n")
    
f.write("\n")   
f.write("Time taken is::::  ")
f.write(str(time_parsing))
f.write("\n")        
f.write("Time taken is::::  ")
f.write(str(time_model))
f.write("\n")

f.write("The output is::::  ")
f.write("\n")
f.write(str(output))
f.write("\n")

f.close()
